In [3]:
from BCBio import GFF
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [4]:
genomeFile = 'alignment/genomeBuild/NZ_CP022438_S.peucetius_subsp.caesius_ATCC_27952_chromosome.gb'
g = SeqIO.read(genomeFile, 'genbank')

In [5]:
type(g)

Bio.SeqRecord.SeqRecord

TODO: we can extract more feature types like rRNA and then filter after the counting has been done.
Try the group features function to in featureCounts program (meta-feature level summarization)

In [11]:
# extract only genes to write to gff file
onlyGenesRec = SeqRecord(g.seq, id=g.id, name='NZ_CP022438')
genes = []
for feat in g.features:
    if feat.type == 'gene':
        genes.append(feat)
onlyGenesRec.features.extend(genes)

In [12]:
with open('alignment/genomeBuild/NZ_CP022438_S.peucetius_subsp.caesius_ATCC_27952_chromosome.gff', 'w') as f:
    GFF.write([onlyGenesRec], f)

In [17]:
import BCBio

In [18]:
dir(BCBio)

['GFF',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

Convert GFF to GTF

locus_tag=CGZ69_RS00005;old_locus_tag=CGZ69_00005

locus_tag "CGZ69_RS00005";old_locus_tag "CGZ69_00005"

In [23]:
with open('alignment/genomeBuild/NZ_CP022438_S.peucetius_subsp.caesius_ATCC_27952_chromosome.gff', 'r') as s:
    with open('alignment/genomeBuild/NZ_CP022438_S.peucetius_subsp.caesius_ATCC_27952_chromosome.gtf', 'w') as t:
        for l in s.readlines():
            if l.startswith('##'):
                continue
            l = l.strip()
            elements = l.split('\t')
            rawAttributes = [e.split('=') for e in elements[-1].split(';')]
            gtfAttrs = ';'.join([f'{a} "{v}"' for a,v in rawAttributes])
            t.write('\t'.join(elements[:-1]+[gtfAttrs])+'\n')
            

Count rRNA

In [25]:
rrnas = []
for feat in g.features:
    if feat.type == 'rRNA':
        rrnas.append(feat.qualifiers['locus_tag'][0])
for r in rrnas:
    print(r)

CGZ69_RS05490
CGZ69_RS05495
CGZ69_RS05500
CGZ69_RS07735
CGZ69_RS07740
CGZ69_RS07745
CGZ69_RS13750
CGZ69_RS13755
CGZ69_RS13760
CGZ69_RS17095
CGZ69_RS17100
CGZ69_RS17105
CGZ69_RS20320
CGZ69_RS20325
CGZ69_RS20330
CGZ69_RS26990
CGZ69_RS26995
CGZ69_RS27000
